At the end of this notebook, we will have submitted to the ["Paddy Doctor: Paddy Disease Classification" competition on Kaggle](https://www.kaggle.com/competitions/paddy-disease-classification/overview)!

Let us begin by downloading the data.

In [1]:
%%bash

exec bash
rm -rf data
mkdir data
apt install unzip


Reading package lists...
Building dependency tree...
Reading state information...
unzip is already the newest version (6.0-21ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.


In [1]:
pip install -U timm==0.6.2dev

     |████████████████████████████████| 497 kB 26.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
!cd data && kaggle competitions download -c paddy-disease-classification && unzip -q paddy-disease-classification.zip

100%|██████████████████████████████████████| 1.02G/1.02G [00:47<00:00, 23.5MB/s]
100%|██████████████████████████████████████| 1.02G/1.02G [00:47<00:00, 22.9MB/s]


In [1]:
import timm

We have now downloaded and extracted the data to the `data` directory.

In [2]:
ls data

sample_submission.csv  submissions/  test_images/  train.csv  train_images/


In [3]:
ls data/train_images

bacterial_leaf_blight/     hispa/
bacterial_leaf_streak/     normal/
bacterial_panicle_blight/  tungro/
blast/                     vit_small_patch16_224_squishvitsmall
brown_spot/                vit_small_patch32_224_squishvitsmall
dead_heart/                vit_small_resnet26d_224_squishvitsmallrs
downy_mildew/


In [4]:
ls data/test_images | head

200001.jpg
200002.jpg
200003.jpg
200004.jpg
200005.jpg
200006.jpg
200007.jpg
200008.jpg
200009.jpg
200010.jpg
ls: write error: Broken pipe


Seems that the train data is organized by directories, with the name of the directory being the label.

Test images just live in `data/test_images`

Let us see what is the format of the sample submission file to have a full picture.

In [5]:
import pandas as pd

sample_sub = pd.read_csv('data/sample_submission.csv')
sample_sub.head()

,image_id,label
0,200001.jpg,NaN
1,200002.jpg,NaN
2,200003.jpg,NaN
3,200004.jpg,NaN
4,200005.jpg,NaN


Mhmm. Guessing the labels for the submission are the names of the directories.

Ok, let's start training!

In [6]:
from fastai.vision.all import *
from fastcore.parallel import *

In [7]:
path = Path('data')
trn_path= path/'train_images'
tst_files = get_image_files(path/'test_images').sorted()

In [8]:
trn_path

Path('data/train_images')

In [9]:
tta_res = []

In [10]:
def train(desc, arch, item, batch, accum=False):
    kwargs = {'bs':32} if accum else{}
    dls = ImageDataLoaders.from_folder(trn_path, seed=42, valid_pct=0.2, item_tfms=item, batch_tfms=batch, **kwargs)
    cbs = GradientAccumulation(2) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    learn.fine_tune(22, 0.02)
    tta_res.append(learn.tta(dl=dls.test_dl(tst_files)))
    learn.export(f'{arch}_{desc}')
    

In [11]:
timm.list_models("convnext_large*")

['convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k']

In [12]:
arch = 'convnext_large_in22k'

In [13]:
train('Convnext', arch, item=Resize(480, method='squish'), batch=aug_transforms(size=224, min_scale=0.75), accum=False)

Could not do one pass in your dataloader, there is something wrong in it. Please see the stack trace below:


RuntimeError: cusolver error: CUSOLVER_STATUS_EXECUTION_FAILED, when calling `cusolverDnSgetrf( handle, m, n, dA, ldda, static_cast<float*>(dataPtr.get()), ipiv, info)`

In [32]:
save_pickle('tta_res.pkl', tta_res)

In [37]:
tta_res

[(TensorBase([[1.1382e-07, 4.4558e-10, 1.7501e-08,  ..., 1.0000e+00, 2.7908e-07,
           4.7529e-09],
          [1.2972e-06, 2.5286e-07, 5.0332e-08,  ..., 2.3146e-07, 1.0000e+00,
           4.5895e-08],
          [1.9073e-06, 1.4750e-08, 5.6766e-06,  ..., 3.2842e-05, 3.9817e-04,
           2.0156e-07],
          ...,
          [2.4073e-08, 4.0051e-08, 1.6932e-06,  ..., 1.8608e-07, 9.9999e-01,
           2.1930e-08],
          [1.1173e-07, 9.9720e-01, 1.7964e-07,  ..., 2.5631e-03, 5.7499e-06,
           6.2488e-06],
          [1.0303e-14, 3.5787e-16, 3.2132e-11,  ..., 4.2362e-12, 1.4127e-12,
           4.1721e-13]]),
  None)]

In [38]:
tta_res=load_pickle('tta_res.pkl')

In [13]:
timm.list_models('vit*')

['vit_base_patch8_224',
 'vit_base_patch8_224_dino',
 'vit_base_patch8_224_in21k',
 'vit_base_patch16_18x2_224',
 'vit_base_patch16_224',
 'vit_base_patch16_224_dino',
 'vit_base_patch16_224_in21k',
 'vit_base_patch16_224_miil',
 'vit_base_patch16_224_miil_in21k',
 'vit_base_patch16_224_sam',
 'vit_base_patch16_384',
 'vit_base_patch16_plus_240',
 'vit_base_patch16_rpn_224',
 'vit_base_patch32_224',
 'vit_base_patch32_224_in21k',
 'vit_base_patch32_224_sam',
 'vit_base_patch32_384',
 'vit_base_patch32_plus_256',
 'vit_base_r26_s32_224',
 'vit_base_r50_s16_224',
 'vit_base_r50_s16_224_in21k',
 'vit_base_r50_s16_384',
 'vit_base_resnet26d_224',
 'vit_base_resnet50_224_in21k',
 'vit_base_resnet50_384',
 'vit_base_resnet50d_224',
 'vit_giant_patch14_224',
 'vit_gigantic_patch14_224',
 'vit_huge_patch14_224',
 'vit_huge_patch14_224_in21k',
 'vit_large_patch14_224',
 'vit_large_patch16_224',
 'vit_large_patch16_224_in21k',
 'vit_large_patch16_384',
 'vit_large_patch32_224',
 'vit_large_patch

In [11]:
arch = 'vit_small_resnet26d_224'

In [ ]:
train('squishvitsmallrs', arch, item=Resize(480, method='squish'), batch=aug_transforms(size=224, min_scale=0.75), accum=False)

No pretrained weights exist or were found for this model. Using random initialization.


epoch,train_loss,valid_loss,error_rate,time
0,2.062253,1.513366,0.480058,00:40


epoch,train_loss,valid_loss,error_rate,time
0,1.124381,0.704420,0.223450,00:45
1,0.740127,0.517468,0.170591,00:45
2,0.592722,0.488723,0.156175,00:45
3,0.482950,0.413550,0.132148,00:45
4,0.412479,0.324986,0.095147,00:45
5,0.353200,0.330423,0.095627,00:45
6,0.289737,0.268565,0.077847,00:44
7,0.245875,0.254864,0.068236,00:45
8,0.219796,0.204995,0.060548,00:45
9,0.163564,0.202536,0.052859,00:45


In [ ]:
save_pickle('tta_res_post_vit4.pkl', tta_res)

In [67]:
tta_res=load_pickle('tta_res_post_vit4.pkl')

In [12]:
timm.list_models('swinv2*')

['swinv2_base_window8_256',
 'swinv2_base_window12_192_22k',
 'swinv2_base_window12to16_192to256_22kft1k',
 'swinv2_base_window12to24_192to384_22kft1k',
 'swinv2_base_window16_256',
 'swinv2_cr_base_224',
 'swinv2_cr_base_384',
 'swinv2_cr_base_ns_224',
 'swinv2_cr_giant_224',
 'swinv2_cr_giant_384',
 'swinv2_cr_huge_224',
 'swinv2_cr_huge_384',
 'swinv2_cr_large_224',
 'swinv2_cr_large_384',
 'swinv2_cr_small_224',
 'swinv2_cr_small_384',
 'swinv2_cr_small_ns_224',
 'swinv2_cr_tiny_224',
 'swinv2_cr_tiny_384',
 'swinv2_cr_tiny_ns_224',
 'swinv2_large_window12_192_22k',
 'swinv2_large_window12to16_192to256_22kft1k',
 'swinv2_large_window12to24_192to384_22kft1k',
 'swinv2_small_window8_256',
 'swinv2_small_window16_256',
 'swinv2_tiny_window8_256',
 'swinv2_tiny_window16_256']

In [12]:
arch = 'swinv2_base_window12_192_22k'

In [13]:
train('squishswin2192', arch, item=Resize(480, method='squish'), batch=aug_transforms(size=192, min_scale=0.75), accum=True)

/opt/conda/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272168290/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


epoch,train_loss,valid_loss,error_rate,time
0,1.401071,0.953908,0.270543,01:39


epoch,train_loss,valid_loss,error_rate,time
0,0.593058,0.314054,0.105718,01:58
1,0.428402,0.261258,0.086016,01:58
2,0.432363,0.346940,0.107160,01:58
3,0.379686,0.417443,0.123018,01:58
4,0.374332,0.267669,0.088900,01:59
5,0.346305,0.427857,0.115810,01:58
6,0.329212,0.260587,0.076406,01:59
7,0.285558,0.281012,0.079289,01:58
8,0.229152,0.221352,0.064392,01:58
9,0.237098,0.205832,0.062470,01:58


In [14]:
save_pickle('tta_res_post_swin2.pkl', tta_res)
# tta_prs = first(zip(*tta_res))

In [46]:
tta_res=load_pickle('tta_res_post_swin2.pkl')

In [68]:
tta_prs = first(zip(*tta_res))

In [69]:
tta_res

[(TensorBase([[1.1382e-07, 4.4558e-10, 1.7501e-08,  ..., 1.0000e+00, 2.7908e-07,
           4.7529e-09],
          [1.2972e-06, 2.5286e-07, 5.0332e-08,  ..., 2.3146e-07, 1.0000e+00,
           4.5895e-08],
          [1.9073e-06, 1.4750e-08, 5.6766e-06,  ..., 3.2842e-05, 3.9817e-04,
           2.0156e-07],
          ...,
          [2.4073e-08, 4.0051e-08, 1.6932e-06,  ..., 1.8608e-07, 9.9999e-01,
           2.1930e-08],
          [1.1173e-07, 9.9720e-01, 1.7964e-07,  ..., 2.5631e-03, 5.7499e-06,
           6.2488e-06],
          [1.0303e-14, 3.5787e-16, 3.2132e-11,  ..., 4.2362e-12, 1.4127e-12,
           4.1721e-13]]),
  None),
 (TensorImage([[3.9709e-05, 5.8992e-06, 3.1102e-06,  ..., 9.9931e-01, 5.1145e-04,
           3.5382e-05],
          [9.8009e-08, 1.9103e-07, 5.3287e-07,  ..., 2.4058e-06, 9.9999e-01,
           2.7695e-07],
          [1.9617e-06, 4.8226e-06, 4.3419e-06,  ..., 5.0541e-05, 2.4748e-05,
           1.2588e-05],
          ...,
          [3.2699e-08, 2.2303e-08, 4.6868

In [72]:
tta_prs += tta_prs[0:1]

In [61]:
tta_prs += tta_prs[1:2]

In [19]:
tta_prs += tta_prs[3:4]

In [73]:
t_tta = torch.stack(tta_prs)

In [74]:
avg_pr = t_tta.mean(0)
idxs = avg_pr.argmax(dim=1)
idxs.shape

torch.Size([3469])

In [75]:
dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=Resize(224))
mapping = dict(enumerate(dls.vocab))
ss = pd.read_csv('data/sample_submission.csv')
results = pd.Series(idxs.numpy(), name='idxs').map(mapping)
ss.label = results
ss.to_csv('data/submissions/subm.csv', index=False)

In [76]:
ss

,image_id,label
0,200001.jpg,hispa
1,200002.jpg,normal
2,200003.jpg,blast
3,200004.jpg,blast
4,200005.jpg,blast
...,...,...
3464,203465.jpg,dead_heart
3465,203466.jpg,hispa
3466,203467.jpg,normal
3467,203468.jpg,bacterial_leaf_streak


In [77]:
!kaggle competitions submit -c paddy-disease-classification -f data/submissions/subm.csv -m "11th Mix more conv no swinv2"

100%|███████████████████████████████████████| 70.4k/70.4k [00:00<00:00, 160kB/s]
Successfully submitted to Paddy Doctor: Paddy Disease Classification

In [37]:
!kaggle competitions submit -c paddy-disease-classification -f data/submissions/subm.csv -m "M1 chip entry"

100%|██████████████████████████████████████| 70.1k/70.1k [00:02<00:00, 32.0kB/s]
Successfully submitted to Paddy Doctor: Paddy Disease Classification